Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/ml-frameworks/pytorch/distributed-pytorch-with-horovod/distributed-pytorch-with-horovod.png)

# Distributed PyTorch with DistributedDataParallel
In this tutorial, you will train a PyTorch model on the [CIFAR10](http://www.cs.toronto.edu/~kriz/cifar.html) dataset using distributed training with PyTorch's `DistributedDataParallel` module across a Azure Stack Hub CPU Kubernetes cluster.

## Prerequisites

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [AZML-SDK-INSTALL](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  to install the Azure Machine Learning Python SDK and create an Azure ML `Workspace`

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [ ]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

## Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`. 

If you haven't done already please go to `config.json` file and fill in your workspace information.

In [ ]:
from azureml.core.workspace import Workspace,  ComputeTarget
from azureml.exceptions import ComputeTargetException

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Prepare dataset

Here we download cifar10 dataset from [cifar10-data](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz). The downloaded data is then registered as dataset in a data store of the workspace. 

To set up datastore using an azure stack hub storage account, please refer to [Train_azure_arc](https://github.com/penorouzi/AML-Kubernetes/blob/master/docs/ASH/Train-AzureArc.md#create-and-configure-azure-stack-hubs-storage-account). 

To register the dataset manually, please refer to this [video](https://msit.microsoftstream.com/video/51f7a3ff-0400-b9eb-2703-f1eb38bc6232)


In [ ]:
import tempfile
import os
import requests
from azureml.core import Dataset

from azureml.exceptions import UserErrorException
dataset_name = 'CIFAR-10'
try:
    ds = Dataset.get_by_name(ws, name=dataset_name)
except UserErrorException as e:#dataset not registered

    path = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'

    with tempfile.TemporaryDirectory() as tmpdir:
        os.makedirs(os.path.join(tmpdir, 'cifar-10'))

        data = requests.get(path, allow_redirects=True).content
        with open(os.path.join(tmpdir, 'cifar-10', path.split('/')[-1]), 'wb') as f:
            f.write(data)
    
        datastore_name = "<my_data_store>"
        ds = Dataset.File.upload_directory(tmpdir, ws.datastores.get(datastore_name), overwrite=True)
        ds.register(ws, dataset_name, 'CIFAR-10 images from https://www.cs.toronto.edu/~kriz/cifar.html')
        

## Create or attach existing ArcKubernetesCompute

You will need to create a [compute target and attach it to AML](https://github.com/penorouzi/AML-Kubernetes/blob/master/docs/ASH/AML-ARC-Compute.md) for training your model. In this tutorial, ArcKubernetesCompute  ArcKubernetesCompute for our remote training compute resource. Make sure azureml-contrib is installed by following [Private Preview branch of AzureML SDK](https://github.com/penorouzi/AML-Kubernetes/blob/master/docs/ASH/AML-ARC-Compute.md#python-sdk-recommended) 

In [ ]:
from azureml.contrib.core.compute.arckubernetescompute import ArcKubernetesCompute

resource_id =  "<arc_cluster_resource_id>"

attach_config = ArcKubernetesCompute.attach_configuration(
    resource_id= resource_id,
)

try:
    attach_name = "arc_attach"
    arcK_target = ArcKubernetesCompute.attach(ws, attach_name, attach_config)
    arcK_target.wait_for_completion(show_output=True)
    print('arc attach  success')
except ComputeTargetException as e:
    print(e)
    print('arc attach  failed')


## Train model on the remote compute
Now that we have the ArcKubernetesCompute ready to go, let's run our distributed training job.

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [ ]:
import os

project_folder = './pytorch-distr'
os.makedirs(project_folder, exist_ok=True)

### Prepare training script
Now you will need to create your training script. In this tutorial, the script for distributed training of CIFAR10 is already provided for you at `cifar_dist_main.py`. In practice, you should be able to take any custom PyTorch training script as is and run it with Azure ML without having to modify your code.

Once your script is ready, copy the training script `cifar_dist_main.py` into the project directory.

In [ ]:
import shutil

run_script = 'cifar_dist_main.py'
shutil.copy(run_script, project_folder)

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed PyTorch tutorial. 

In [ ]:
from azureml.core import Experiment

experiment_name = 'pytorch-cifar-distr'
experiment = Experiment(ws, name=experiment_name)

### Create an environment

Define a conda environment YAML file with your training script dependencies and create an Azure ML environment.

In [ ]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-defaults
  - torch==1.6.0
  - torchvision==0.7.0
  - future==0.17.1

In [ ]:
from azureml.core import Environment

pytorch_env = Environment.from_conda_specification(name = 'pytorch-1.6-cpu', file_path = './conda_dependencies.yml')

# Specify a CPU base image

pytorch_env.docker.enabled = True
pytorch_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04'


### Configure the training job: torch.distributed with GLOO backend

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on.

In order to run a distributed PyTorch job with **torch.distributed** using the GLOO backend, create a `PyTorchConfiguration` and pass it to the `distributed_job_config` parameter of the ScriptRunConfig constructor. Specify `communication_backend='Gloo'` in the PyTorchConfiguration. The below code will configure node_count = 2. These is the number of worker nodes. The number of  distributed jobs will be 3 if one master node is used.  GLOO backend which is recommended backend for communications between CPUs.

In [ ]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
from azureml.core import Dataset
import os

dataset_ash = Dataset.get_by_name(ws, name=dataset_name)
args = [
        '--data-folder', dataset_ash.as_mount(),
        '--dist-backend', 'gloo',
           ]

distributed_job_config=PyTorchConfiguration(communication_backend='Gloo', node_count=1) #configuring AML pytorch config


src = ScriptRunConfig(
                     source_directory=project_folder,
                      script=run_script,
                      arguments=args,
                      compute_target=arcK_target,
                      environment=pytorch_env,
                      distributed_job_config=distributed_job_config)

### Submit job
Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.

In [ ]:
run = experiment.submit(src)

In [ ]:
run.wait_for_completion(show_output=True) # this provides a verbose log